## Primer Cuatrimestre de 2017. Examen parcial, primera oportunidad.

Se tiene información estadística de la temporada regular de todos los
jugadores de la NBA en un RDD de tuplas con el siguiente formato:
(id_jugador, nombre, promedio_puntos, promedio_asistencias, promedio_robos,
promedio_bloqueos, promedio_rebotes, promedio_faltas). 

Un analista de la
cadena ESPN está trabajando con un RDD que corresponde a la primera ronda
de playoffs y que tiene el siguiente formato: (id_jugador, id_partido, timestamp,
cantidad_puntos, cantidad_rebotes, cantidad_bloqueos, cantidad_robos,
cantidad_asistencias, cantidad_faltas). 

En base a estos RDDs se quiere
programar en PySpark un programa que genere un RDD con los nombres (sin
duplicados) de los jugadores que lograron en algún partido de playoffs una
cantidad de asistencias mayor a su promedio histórico. (****) (15 pts)

In [26]:
# Criterio

# Existen diversas maneras de realizarlo, 
# pero es posible resolverlo realizando un reduceByKey con clave (id_jugador,id_partido) 
# acumulando la cantidad de asistencias por partido quedando ((id_jugador, id_partido), sum_asistencias).
# Luego efectuamos un cambio de clave a id_jugador con un map para 
# poder realizar un join de la informacion de la temporada usando id_jugador.
# Una vez hecho esto, filtramos aquella informacion 
# que cumple de mayor cantidad asistencias en el partido que el promedio de la temporada.
# Dado que pueden quedarnos duplicados (un jugador podria superar su promedio de temporada en varios partidos) 
# es importante al mapearla informacion al formato final realizar un distinct.

In [1]:
import pyspark

try: 
    type(sc)
except NameError:
    sc = pyspark.SparkContext('local[*]')
    
# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, nombre, promedio_asistencias)
players_all_time_stats = [
    (1, 'Manu Ginobili', 800),
    (2, 'Kobe Bryant', 100),
    (3, 'Marc Gasol', 25),
    (4, 'James Harden', 1000)]

# usamos para simplificar el formato, que puede obtenerse con un map.
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
scores = [
  (1, 1, 1, 100),
  (1, 1, 3, 100),
  (2, 1, 1, 150),
  (2, 1, 3, 150),
  (3, 2, 2, 50),
  (3, 2, 3, 50),      
  (1, 2, 1, 150),
  (1, 2, 3, 150),
]

players_all_time_stats_rdd = sc.parallelize(players_all_time_stats)
scores_rdd = sc.parallelize(scores)

In [2]:
# cambio de clave a id_jugador, partido
scores_by_match = scores_rdd.map(lambda a: ((a[0],a[1]),a[3]))
# scores_by_match = scores_by_match.reduceByKey(lambda a,b: a+b)
scores_by_match.collect()

[((1, 1), 100),
 ((1, 1), 100),
 ((2, 1), 150),
 ((2, 1), 150),
 ((3, 2), 50),
 ((3, 2), 50),
 ((1, 2), 150),
 ((1, 2), 150)]

In [3]:
# cambio de clave a id_jugador
scores_by_player = scores_by_match.map(lambda a: (a[0][0], a[1]))
scores_by_player.collect()

[(1, 100), (1, 100), (2, 150), (2, 150), (3, 50), (3, 50), (1, 150), (1, 150)]

In [5]:
# juntamos los datos para poder evaluar lo pedido
# preparamos la key del lado de informacion historica de jugadores.

all_time_with_key = players_all_time_stats_rdd.map(lambda a: (a[0],a))
all_time_with_key.collect()

[(1, (1, 'Manu Ginobili', 800)),
 (2, (2, 'Kobe Bryant', 100)),
 (3, (3, 'Marc Gasol', 25)),
 (4, (4, 'James Harden', 1000))]

In [6]:
scores_by_player = scores_by_player.join(all_time_with_key)
scores_by_player.collect()

[(1, (100, (1, 'Manu Ginobili', 800))),
 (1, (100, (1, 'Manu Ginobili', 800))),
 (1, (150, (1, 'Manu Ginobili', 800))),
 (1, (150, (1, 'Manu Ginobili', 800))),
 (2, (150, (2, 'Kobe Bryant', 100))),
 (2, (150, (2, 'Kobe Bryant', 100))),
 (3, (50, (3, 'Marc Gasol', 25))),
 (3, (50, (3, 'Marc Gasol', 25)))]

In [19]:
overperformers = scores_by_player.filter(lambda a: a[1][0] > a[1][1][2])
overperformers.collect()

[(2, (150, (2, 'Kobe Bryant', 100))),
 (2, (150, (2, 'Kobe Bryant', 100))),
 (3, (50, (3, 'Marc Gasol', 25))),
 (3, (50, (3, 'Marc Gasol', 25)))]

In [20]:
# veamos los duplicados
overperformers = overperformers.map(lambda a: a[1][1][1]).distinct()
overperformers.take(5)

['Kobe Bryant', 'Marc Gasol']

# Esta es mi resolucion

In [22]:
# (id_jugador, nombre, promedio_asistencias)
promediosJugadores = sc.parallelize(players_all_time_stats)
promediosJugadores.collect()

[(1, 'Manu Ginobili', 800),
 (2, 'Kobe Bryant', 100),
 (3, 'Marc Gasol', 25),
 (4, 'James Harden', 1000)]

In [29]:
# (id_jugador, id_partido, timestamp, cantidad_asistencias)
partidosJugadores = sc.parallelize(scores)
partidosJugadores.collect()

[(1, 1, 1, 100),
 (1, 1, 3, 100),
 (2, 1, 1, 150),
 (2, 1, 3, 150),
 (3, 2, 2, 50),
 (3, 2, 3, 50),
 (1, 2, 1, 150),
 (1, 2, 3, 150)]

In [31]:
# Hago un rdd con clave (id_jugador, id_partido) y valor cantidad_asistencias
partidosJugadoresAgrupados = partidosJugadores.map(lambda x: ((x[0], x[1]), x[3]))
partidosJugadoresAgrupados.collect()

[((1, 1), 100),
 ((1, 1), 100),
 ((2, 1), 150),
 ((2, 1), 150),
 ((3, 2), 50),
 ((3, 2), 50),
 ((1, 2), 150),
 ((1, 2), 150)]

In [33]:
# Vienen datos para el mismo jugador en el mismo partido, entonces los sumo.
partidosJugadoresSuma = partidosJugadoresAgrupados.reduceByKey(lambda x, y: x + y)
partidosJugadoresSuma.collect()

[((1, 1), 200), ((1, 2), 300), ((3, 2), 100), ((2, 1), 300)]

In [45]:
# Hago un nuevo rdd en donde me quede jugador_id como clave y como valor: (partido_id, cantidad_asistencias)
datosPlayoff = partidosJugadoresSuma.map(lambda x: (x[0][0], (x[0][1], x[1])))
datosPlayoff.collect()

[(1, (1, 200)), (1, (2, 300)), (3, (2, 100)), (2, (1, 300))]

In [46]:
# Preparo los datos historiscos para un join.
datosHistoricos = players_all_time_stats_rdd.map(lambda x: (x[0], (x[1], x[2])))
datosHistoricos.collect()

[(1, ('Manu Ginobili', 800)),
 (2, ('Kobe Bryant', 100)),
 (3, ('Marc Gasol', 25)),
 (4, ('James Harden', 1000))]

In [48]:
# Hago el join entre los datos historicos y los datos de los playoff.
datosFinales = datosPlayoff.join(datosHistoricos)
datosFinales.collect()

[(1, ((1, 200), ('Manu Ginobili', 800))),
 (1, ((2, 300), ('Manu Ginobili', 800))),
 (2, ((1, 300), ('Kobe Bryant', 100))),
 (3, ((2, 100), ('Marc Gasol', 25)))]

In [51]:
# Filtro aquellos cuyas asistencias por partido superen el promedio.
mejoresAsistencias = datosFinales.filter(lambda x: x[1][0][1] > x[1][1][1])
mejoresAsistencias.collect()

[(2, ((1, 300), ('Kobe Bryant', 100))), (3, ((2, 100), ('Marc Gasol', 25)))]

In [55]:
# Me quedo solo con los datos de los nombres de los jugadores.
nombresJugadores = mejoresAsistencias.map(lambda x: x[1][1][0])
nombresJugadores.take(10)

['Kobe Bryant', 'Marc Gasol']